In [ ]:
import import_ipynb
from AlexFunctions import plot_images

import cv2
import numpy as np
import os

In [ ]:
def get_mission_file_path(mission_number):
    return '..\\missions\\mission_' + str(mission_number) + '\\mission_' + str(mission_number) + '_'

def get_global_image(mission_number, get_normalized=False):
    if get_normalized:
        mission = np.load(get_mission_file_path(mission_number) + 'normalized_image.npy')
    else:
        mission = np.load(get_mission_file_path(mission_number) + 'aligned_image.npy')
    
    return mission

def get_region_and_sample_image(mission_number, region_name, get_normalized=False):
    if get_normalized:
        region = np.load(get_mission_file_path(mission_number) + 'normalized_region_image_' + region_name + '.npy')
        sample = np.load(get_mission_file_path(mission_number) + 'normalized_sample_image_' + region_name + '.npy')
    else:
        region = np.load(get_mission_file_path(mission_number) + 'aligned_region_image_' + region_name + '.npy')
        sample = np.load(get_mission_file_path(mission_number) + 'aligned_sample_image_' + region_name + '.npy')
    
    return region, sample

In [ ]:
def calculate_cdf(histogram):
    """
    This method calculates the cumulative distribution function
    :param array histogram: The values of the histogram
    :return: normalized_cdf: The normalized cumulative distribution function
    :rtype: array
    """
    # Get the cumulative sum of the elements
    cdf = histogram.cumsum()
 
    # Normalize the cdf
    normalized_cdf = cdf / float(cdf.max())
 
    return normalized_cdf
 
def calculate_lookup(new_cdf, ref_cdf):
    """
    This method creates the lookup table
    :param array new_cdf: The cdf for the source image
    :param array ref_cdf: The cdf for the reference image
    :return: lookup_table: The lookup table
    :rtype: array
    """
    lookup_table = np.zeros(256)
    lookup_val = 0
    for new_pixel_val in range(len(new_cdf)):
        lookup_val
        for ref_pixel_val in range(len(ref_cdf)):
            if ref_cdf[ref_pixel_val] >= new_cdf[new_pixel_val]:
                lookup_val = ref_pixel_val
                break
        lookup_table[new_pixel_val] = lookup_val
    return lookup_table

def match_histograms(ref_image, new_image):
    """
    This method matches the new image histogram to the
    reference signal
    :param image new_image: The original source image
    :param image  ref_image: The reference image
    :return: image_after_matching
    :rtype: image (array)
    """
    # Split the images into the different color channels
    # b means blue, g means green and r means red
    new_b, new_g, new_r = cv2.split(new_image)
    ref_b, ref_g, ref_r = cv2.split(ref_image)
 
    # Compute the b, g, and r histograms separately
    # The flatten() Numpy method returns a copy of the array c
    # collapsed into one dimension.
    new_hist_blue, bin_0 = np.histogram(new_b.flatten(), 256, [0,256])
    new_hist_green, bin_1 = np.histogram(new_g.flatten(), 256, [0,256])
    new_hist_red, bin_2 = np.histogram(new_r.flatten(), 256, [0,256])    
    ref_hist_blue, bin_3 = np.histogram(ref_b.flatten(), 256, [0,256])    
    ref_hist_green, bin_4 = np.histogram(ref_g.flatten(), 256, [0,256])
    ref_hist_red, bin_5 = np.histogram(ref_r.flatten(), 256, [0,256])
 
    # Compute the normalized cdf for the source and reference image
    new_cdf_blue = calculate_cdf(new_hist_blue)
    new_cdf_green = calculate_cdf(new_hist_green)
    new_cdf_red = calculate_cdf(new_hist_red)
    ref_cdf_blue = calculate_cdf(ref_hist_blue)
    ref_cdf_green = calculate_cdf(ref_hist_green)
    ref_cdf_red = calculate_cdf(ref_hist_red)

    # Make a separate lookup table for each color
    blue_lookup_table = calculate_lookup(new_cdf_blue, ref_cdf_blue)
    green_lookup_table = calculate_lookup(new_cdf_green, ref_cdf_green)
    red_lookup_table = calculate_lookup(new_cdf_red, ref_cdf_red)
 
    # Use the lookup function to transform the colors of the original
    # source image
    blue_after_transform = cv2.LUT(new_b, blue_lookup_table)
    green_after_transform = cv2.LUT(new_g, green_lookup_table)
    red_after_transform = cv2.LUT(new_r, red_lookup_table)
 
    # Put the image back together
    image_after_matching = cv2.merge([blue_after_transform, green_after_transform, red_after_transform])
    image_after_matching = cv2.convertScaleAbs(image_after_matching)
 
    return image_after_matching

In [ ]:
def run_load_data(mission_numbers, debug_images=False):   
    print('Loading Missions:', mission_numbers)
    
    mission_images_bgr = []
    grass_images_bgr = []
    building_images_bgr = []

    for mission_number in mission_numbers:
        if not os.path.isdir('..\\missions\\mission_' + str(mission_number)):
            print('[Error] Mission %d directory has not been created' % (mission_number))
            break

        mission_image = get_global_image(mission_number)
        mission_images_bgr.append(mission_image)

        grass_image, _ = get_region_and_sample_image(mission_number, 'GRASS')
        grass_images_bgr.append(grass_image)
        building_image, _ = get_region_and_sample_image(mission_number, 'BUILDING')
        building_images_bgr.append(building_image)
        
    if debug_images:
        plot_images(mission_images_bgr, ['mission_images_bgr ' + str(mission_number) for mission_number in mission_numbers])
        plot_images(grass_images_bgr, ['grass_images_bgr ' + str(mission_number) for mission_number in mission_numbers])
        plot_images(building_images_bgr, ['building_images_bgr ' + str(mission_number) for mission_number in mission_numbers])
    
    histogram_mission_images_bgr = []
    histogram_grass_images_bgr = []
    histogram_building_images_bgr = []

    for i in range(0, len(mission_numbers)):
        histogram_mission_images_bgr.append(match_histograms(mission_images_bgr[0], mission_images_bgr[i]))
        histogram_grass_images_bgr.append(match_histograms(grass_images_bgr[0], grass_images_bgr[i]))
        histogram_building_images_bgr.append(match_histograms(building_images_bgr[0], building_images_bgr[i]))
        
    if debug_images:
        plot_images(histogram_mission_images_bgr, ['histogram_mission_images_bgr ' + str(mission_number) for mission_number in mission_numbers])
        plot_images(histogram_grass_images_bgr, ['histogram_grass_images_bgr ' + str(mission_number) for mission_number in mission_numbers])
        plot_images(histogram_building_images_bgr, ['histogram_building_images_bgr ' + str(mission_number) for mission_number in mission_numbers])
    
    mission_images_hsv = [] # used for shadow detection
    histogram_mission_images_hsv = []
    histogram_grass_images_hsv = []
    histogram_building_images_hsv = []

    for i in range(0, len(mission_numbers)):
        hsv = cv2.cvtColor(mission_images_bgr[i].copy(), cv2.COLOR_BGR2HSV)
        mission_images_hsv.append(hsv)

        hsv = cv2.cvtColor(histogram_mission_images_bgr[i].copy(), cv2.COLOR_BGR2HSV)
        histogram_mission_images_hsv.append(hsv)

        hsv = cv2.cvtColor(histogram_grass_images_bgr[i].copy(), cv2.COLOR_BGR2HSV)
        histogram_grass_images_hsv.append(hsv)

        hsv = cv2.cvtColor(histogram_building_images_bgr[i].copy(), cv2.COLOR_BGR2HSV)
        histogram_building_images_hsv.append(hsv)
    
    if debug_images:
        plot_images(mission_images_hsv, ['mission_images_hsv ' + str(mission_number) for mission_number in mission_numbers])
        plot_images(histogram_mission_images_hsv, ['histogram_mission_images_hsv ' + str(mission_number) for mission_number in mission_numbers])
        plot_images(histogram_grass_images_hsv, ['histogram_grass_images_hsv ' + str(mission_number) for mission_number in mission_numbers])
        plot_images(histogram_building_images_hsv, ['histogram_building_images_hsv ' + str(mission_number) for mission_number in mission_numbers])
    
    print('Data Loaded')
    
    return [mission_images_bgr, grass_images_bgr, building_images_bgr, histogram_mission_images_bgr, histogram_grass_images_bgr, histogram_building_images_bgr, mission_images_hsv, histogram_mission_images_hsv, histogram_grass_images_hsv, histogram_building_images_hsv]